# COVID-19 and Weather Patterns

## Imports

- *os* for interfacing with the operating system
- *pathlib* for interfacing with the file system
- *zipfile* for managing archive files

- *numpy* for array processing
- *pandas* for tabular processing
- *tensorflow* for tensor processing
- *keras* for simplified tensor processing

- *matplotlib* for visualization
- *seaborn* for enhanced visualization

In [1]:
# Custom
import data_processing

# File System
import os
from pathlib import Path
from zipfile import ZipFile

# Processing
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

tensorflow.__version__

'2.7.0'

## Introduction

The goal of this work is to determine whether or not weather patterns should be considered as a meaningful supporting input when making predictions about new daily COVID-19 cases within a given geographical space. Using census, weather, and COVID-19 datasets provided by the Urban Sustain project, the authors attempt to quantify the correlation between particular weather patterns and COVID-19 transmission events.

## Defining Terms

***Urban Sustain Project*** - A joint effort between researchers at Colorado State University, Arizona State University, the University of California-Irvine, and the University of Maryland – Baltimore County.

## Loading Data

**DEVELOPER NOTE:** Download the five required datasets from Urban Sustain and place them in the cloned repository at ```./data/```. These datasets are also available at a shared OneDrive folder. This logic expects that these files exist at relative path ```../data/``` with respect to this notebook.

We'll begin by defining a path to our data directory and a list of the datasets that we expect to find there.

In [2]:
DATA_PATH = '../data/' # Point this constant to the location of your data archive files

EXPECTED_DATASETS = [
    'county_total_population.Colorado.zip',
    'covid_county.Colorado.zip',
    'neon_2d_wind.Colorado.zip',
    'neon_barometric_pressure.Colorado.zip',
    'neon_single_asp_air_temperature.Colorado.zip'
]

Next, we will attempt to extract each of these archived datasets into a dedicated subdirectory within the given data directory.

In [3]:
# For each listed dataset string in the EXPECTED_DATASETS constant
for datasetName in EXPECTED_DATASETS:
    try:
        # Open the given archive file
        with ZipFile(DATA_PATH + datasetName, 'r') as currentZip:
            # Build the target directory path for extracted data
            datasetNameTokens = datasetName.split('.')
            datasetNameTokens.remove('zip')
            targetDirectory = DATA_PATH + '.'.join(datasetNameTokens)
            
            # If the target directory doesn't exist, create it
            if not os.path.exists(targetDirectory):
                Path(targetDirectory).mkdir()
            
            # Extract all data from the archive file to the target directory
            currentZip.extractall(targetDirectory)
    except FileNotFoundError:
        print("Unable to open " + datasetName + " at path " + DATA_PATH + datasetName)

## File Information

Four of the five datasets referenced in this work relate geospatial information to particular events as they occur over time Each is provided by the Urban Sustain project and employs a similar file structure.

- data.json
- fieldLabels.json
- linkedGeometry.json
- README.txt


#### New York Times COVID-19 County Dataset

The included COVID-19 County dataset 

In [4]:
covidCountyDataFrame = pd.io.json.read_json('../data/covid_county.Colorado/data.json')
covidCountyDataFrame.sample(5)

,_id,GISJOIN,epoch_time,dateString,county,state,totalCaseCount,newCaseCount,totalDeathCount,newDeathCount
15216,{'$oid': '615fc625bf06594c4cad7e8a'},G0800470,{'$numberLong': '1616133600000'},3/19/21,Gilpin,Colorado,182,1,3,0
17679,{'$oid': '615fc626bf06594c4cad8838'},G0800550,{'$numberLong': '1614582000000'},3/1/21,Huerfano,Colorado,310,1,16,0
15101,{'$oid': '615fc625bf06594c4cad7d24'},G0800470,{'$numberLong': '1585375200000'},3/28/20,Gilpin,Colorado,0,0,0,0
34469,{'$oid': '615fc627bf06594c4cadcb1f'},G0801090,{'$numberLong': '1584770400000'},3/21/20,Saguache,Colorado,0,0,0,0
31536,{'$oid': '615fc627bf06594c4cadc021'},G0800990,{'$numberLong': '1611212400000'},1/21/21,Prowers,Colorado,1088,0,21,1


In [5]:
covidCountyLabelsFrame = pd.io.json.read_json('../data/covid_county.Colorado/fieldLabels.json')
covidCountyLabelsFrame.sample(5)

,name,label
1,newDeathCount,New Death Count
3,newCaseCount,New Case Count
2,totalDeathCount,Total Death Count
0,epoch_time,Date
4,totalCaseCount,Total Case Count


In [6]:
covidCountyGeometryFrame = pd.io.json.read_json('../data/covid_county.Colorado/linkedGeometry.json')
covidCountyGeometryFrame.sample(5)

,_id,type,geometry,properties,GISJOIN
24,{'$oid': '604964eefed2e95d6b68b9d8'},Feature,"{'type': 'Polygon', 'coordinates': [[[-105.397...","{'STATEFP10': '08', 'COUNTYFP10': '047', 'COUN...",G0800470
53,{'$oid': '604964eefed2e95d6b68b837'},Feature,"{'type': 'Polygon', 'coordinates': [[[-106.038...","{'STATEFP10': '08', 'COUNTYFP10': '105', 'COUN...",G0801050
7,{'$oid': '604964eefed2e95d6b68b9c4'},Feature,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...","{'STATEFP10': '08', 'COUNTYFP10': '014', 'COUN...",G0800140
41,{'$oid': '604964eefed2e95d6b68b841'},Feature,"{'type': 'Polygon', 'coordinates': [[[-107.918...","{'STATEFP10': '08', 'COUNTYFP10': '081', 'COUN...",G0800810
44,{'$oid': '604964ecfed2e95d6b68b67a'},Feature,"{'type': 'Polygon', 'coordinates': [[[-103.581...","{'STATEFP10': '08', 'COUNTYFP10': '087', 'COUN...",G0800870


#### NEON 2D Wind Dataset

In [7]:
windDataFrame = pd.io.json.read_json('../data/neon_2d_wind.Colorado/data.json')
windDataFrame.sample(5)

,_id,site,epoch_time,startDateTime,endDateTime,windSpeedMean,windSpeedMinimum,windSpeedMaximum,windSpeedVariance,windSpeedNumPts,windSpeedExpUncert,windSpeedStdErMean,windSpeedFinalQF,windDirMean,windDirVariance,windDirNumPts,windDirExpUncert,windDirStdErMean,windDirFinalQF
865356,{'$oid': '60cbc29651c1683b46654082'},RMNP_DP1.00001.001_000.020,{'$numberLong': '1561914000000'},2019-06-30T17:00:00Z,2019-06-30T17:30:00Z,0.72,0.02,2.33,0.13,1795,0.05,0.01,0,125.86,2157.77,1795,0.07,0.02,1
1279319,{'$oid': '60cbc04551c1683b4624b489'},STER_DP1.00001.001_000.020,{'$numberLong': '1496406600000'},2017-06-02T12:30:00Z,2017-06-02T13:00:00Z,0.79,0.00,2.52,0.24,1795,0.05,0.01,0,250.54,5324.63,1795,0.19,0.03,0
1348391,{'$oid': '60cbc03051c1683b462282e8'},STER_DP1.00001.001_000.030,{'$numberLong': '1396141200000'},2014-03-30T01:00:00Z,2014-03-30T01:30:00Z,1.15,0.22,1.58,0.11,1798,0.03,0.01,0,37.16,409.34,1798,0.19,0.01,0
795242,{'$oid': '60cbc26f51c1683b4660e466'},RMNP_DP1.00001.001_000.020,{'$numberLong': '1419636600000'},2014-12-26T23:30:00Z,2014-12-27T00:00:00Z,0.17,0.00,0.80,0.01,1800,0.02,0.00,0,114.25,3634.06,1800,0.07,0.03,1
499695,{'$oid': '60cbc08b51c1683b462c6929'},CPER_DP1.00001.001_000.030,{'$numberLong': '1590609600000'},2020-05-27T20:00:00Z,2020-05-27T20:30:00Z,6.17,2.46,13.62,3.17,1798,0.60,0.04,0,319.01,280.54,1798,0.07,0.01,0


In [8]:
windLabelsFrame = pd.io.json.read_json('../data/neon_2d_wind.Colorado/fieldLabels.json')
windLabelsFrame.sample(5)

,name,label
25,windSpeedValidCalFailQM,Wind speed quality metric that summarizes the ...
6,windSpeedVariance,Variance in wind speed
48,windDirSensorErrorPassQM,Wind direction quality metric that summarizes ...
45,windDirGapPassQM,Wind direction quality metric that summarizes ...
18,windSpeedNullNAQM,Wind speed quality metric that summarizes when...


In [9]:
windGeometryFrame = pd.io.json.read_json('../data/neon_2d_wind.Colorado/linkedGeometry.json')
windGeometryFrame.sample(5)

,_id,type,geometry,properties,site
13,{'$oid': '60cbcf5eed5577c25c205208'},Feature,"{'type': 'Point', 'coordinates': [-103.02929, ...","{'HOR.VER': '000.010', 'name': 'CFGLOC100113',...",STER_DP1.00001.001_000.010
14,{'$oid': '60cbcf5eed5577c25c205268'},Feature,"{'type': 'Point', 'coordinates': [-102.450966,...","{'HOR.VER': '200.000', 'name': 'CFGLOC101251',...",ARIK_DP1.00001.001_200.000
11,{'$oid': '60cbcf5eed5577c25c20520a'},Feature,"{'type': 'Point', 'coordinates': [-103.02929, ...","{'HOR.VER': '000.030', 'name': 'CFGLOC100115',...",STER_DP1.00001.001_000.030
0,{'$oid': '60cbcf5eed5577c25c205230'},Feature,"{'type': 'Point', 'coordinates': [-105.545955,...","{'HOR.VER': '000.010', 'name': 'CFGLOC100666',...",RMNP_DP1.00001.001_000.010
1,{'$oid': '60cbcf5eed5577c25c205233'},Feature,"{'type': 'Point', 'coordinates': [-105.545955,...","{'HOR.VER': '000.040', 'name': 'CFGLOC100675',...",RMNP_DP1.00001.001_000.040


#### NEON Barometric Pressure Dataset

In [10]:
pressureDataFrame = pd.io.json.read_json('../data/neon_barometric_pressure.Colorado/data.json')
pressureDataFrame.sample(5)

,_id,site,epoch_time,startDateTime,endDateTime,staPresMean,staPresMinimum,staPresMaximum,staPresVariance,staPresNumPts,staPresExpUncert,staPresStdErMean,staPresFinalQF,corPres,corPresExpUncert,corPresFinalQF
358842,{'$oid': '60c78c9fc8cb4f3f3b1b3f6d'},RMNP_DP1.00004.001,{'$numberLong': '1569171600000'},2019-09-22T17:00:00Z,2019-09-22T17:30:00Z,73.14534,73.10517,73.16517,0.00011,180,0.04724,0.00077,0,100.54814,0.39440,0
205373,{'$oid': '60c78b14c8cb4f3f3bedbdde'},CPER_DP1.00004.001,{'$numberLong': '1566093600000'},2019-08-18T02:00:00Z,2019-08-18T02:30:00Z,82.90891,82.87507,82.93508,0.00015,180,0.01730,0.00093,0,99.98879,0.13755,0
382936,{'$oid': '60c78ca2c8cb4f3f3b1b9d10'},RMNP_DP1.00004.001,{'$numberLong': '1613370600000'},2021-02-15T06:30:00Z,2021-02-15T07:00:00Z,71.75059,71.73970,71.75970,0.00002,180,0.04721,0.00030,0,102.54320,0.51241,0
118369,{'$oid': '60c78b08c8cb4f3f3bec6a70'},COMO_DP1.00004.001,{'$numberLong': '1584505800000'},2020-03-18T04:30:00Z,2020-03-18T05:00:00Z,70.04377,70.03711,70.04710,0.00002,180,0.04722,0.00035,0,100.99930,0.52790,0
388314,{'$oid': '60c78cd9c8cb4f3f3b21de7b'},STER_DP1.00004.001,{'$numberLong': '1437154200000'},2015-07-17T17:30:00Z,2015-07-17T18:00:00Z,85.92213,85.91149,85.93050,0.00002,180,0.01721,0.00031,0,99.89649,0.09247,0


In [11]:
pressureLabelsFrame = pd.io.json.read_json('../data/neon_barometric_pressure.Colorado/fieldLabels.json')
pressureLabelsFrame.sample(5)

,name,label
1,startDateTime,Date and time at which a sampling is initiated
0,epoch_time,Date
18,staPresStepNAQM,Station pressure quality metric that summarize...
8,staPresExpUncert,Expanded uncertainty for station pressure
12,staPresRangeNAQM,Station pressure quality metric that summarize...


In [12]:
pressureGeometryFrame = pd.io.json.read_json('../data/neon_barometric_pressure.Colorado/linkedGeometry.json')
pressureGeometryFrame.sample(5)

,_id,type,geometry,properties,site
5,{'$oid': '60c78a9d230c7f49850c20e3'},Feature,"{'type': 'Point', 'coordinates': [-103.02929, ...","{'site': 'STER_DP1.00004.001', 'name': 'neon_b...",STER_DP1.00004.001
4,{'$oid': '60c78a9d230c7f49850c20b1'},Feature,"{'type': 'Point', 'coordinates': [-104.74559, ...","{'site': 'CPER_DP1.00004.001', 'name': 'neon_b...",CPER_DP1.00004.001
1,{'$oid': '60c78a9d230c7f49850c20ef'},Feature,"{'type': 'Point', 'coordinates': [-105.914674,...","{'site': 'WLOU_DP1.00004.001', 'name': 'neon_b...",WLOU_DP1.00004.001
2,{'$oid': '60c78a9d230c7f49850c20b0'},Feature,"{'type': 'Point', 'coordinates': [-105.544602,...","{'site': 'COMO_DP1.00004.001', 'name': 'neon_b...",COMO_DP1.00004.001
3,{'$oid': '60c78a9d230c7f49850c20cf'},Feature,"{'type': 'Point', 'coordinates': [-105.58238, ...","{'site': 'NIWO_DP1.00004.001', 'name': 'neon_b...",NIWO_DP1.00004.001


#### NEON Air Temperature

In [13]:
temperatureDataFrame = pd.io.json.read_json('../data/neon_single_asp_air_temperature.Colorado/data.json')
temperatureDataFrame.sample(5)

,_id,site,epoch_time,startDateTime,endDateTime,tempSingleMean,tempSingleMinimum,tempSingleMaximum,tempSingleVariance,tempSingleNumPts,tempSingleExpUncert,tempSingleStdErMean,finalQF
798319,{'$oid': '60cbd5210fc3818c81080776'},RMNP_DP1.00002.001_000.030,{'$numberLong': '1468996200000'},2016-07-20T06:30:00Z,2016-07-20T07:00:00Z,13.8984,13.3146,14.7402,0.0653,1800,0.1411,0.0060,0
9300,{'$oid': '60cbd73c0fc3818c814785b6'},ARIK_DP1.00002.001_200.000,{'$numberLong': '1480789800000'},2016-12-03T18:30:00Z,2016-12-03T19:00:00Z,4.9248,3.9948,6.0538,0.1928,1800,0.1384,0.0103,0
178183,{'$oid': '60cbd3c40fc3818c81de5120'},CPER_DP1.00002.001_000.010,{'$numberLong': '1461510000000'},2016-04-24T15:00:00Z,2016-04-24T15:30:00Z,9.6705,9.5497,10.2233,0.0187,1800,0.1388,0.0032,0
525620,{'$oid': '60cbd89c0fc3818c8171297a'},NIWO_DP1.00002.001_000.020,{'$numberLong': '1581352200000'},2020-02-10T16:30:00Z,2020-02-10T17:00:00Z,-13.1838,-13.2809,-13.0743,0.0031,1800,0.1283,0.0013,0
668003,{'$oid': '60cbd53c0fc3818c810afcb6'},RMNP_DP1.00002.001_000.010,{'$numberLong': '1576670400000'},2019-12-18T12:00:00Z,2019-12-18T12:30:00Z,-11.9939,-13.1105,-10.3501,0.3838,1800,0.1328,0.0146,0


In [14]:
temperatureLabelsFrame = pd.io.json.read_json('../data/neon_single_asp_air_temperature.Colorado/fieldLabels.json')
temperatureLabelsFrame.sample(5)

,name,label
3,tempSingleMean,Arithmetic mean of single aspirated air temper...
34,validCalFailQM,Quality metric that summarizes the failed outc...
25,spikeFailQM,Quality metric that summarizes the failed outc...
10,rangeFailQM,Quality metric that summarizes the failed outc...
38,betaQM,Quality metric detailing the outcomes of the b...


In [15]:
temperatureGeometryFrame = pd.io.json.read_json('../data/neon_single_asp_air_temperature.Colorado/linkedGeometry.json')
temperatureGeometryFrame.sample(5)

,_id,type,geometry,properties,site
5,{'$oid': '60cbd3907adbc5ff35cc421b'},Feature,"{'type': 'Point', 'coordinates': [-105.544602,...","{'HOR.VER': '200.000', 'name': 'CFGLOC101559',...",COMO_DP1.00002.001_200.000
9,{'$oid': '60cbd3907adbc5ff35cc41a6'},Feature,"{'type': 'Point', 'coordinates': [-104.74559, ...","{'HOR.VER': '000.030', 'name': 'CFGLOC100244',...",CPER_DP1.00002.001_000.030
10,{'$oid': '60cbd3907adbc5ff35cc41a5'},Feature,"{'type': 'Point', 'coordinates': [-104.74559, ...","{'HOR.VER': '000.020', 'name': 'CFGLOC100241',...",CPER_DP1.00002.001_000.020
6,{'$oid': '60cbd3907adbc5ff35cc422f'},Feature,"{'type': 'Point', 'coordinates': [-105.58238, ...","{'HOR.VER': '000.030', 'name': 'CFGLOC109358',...",NIWO_DP1.00002.001_000.030
3,{'$oid': '60cbd3907adbc5ff35cc41c9'},Feature,"{'type': 'Point', 'coordinates': [-105.545955,...","{'HOR.VER': '000.010', 'name': 'CFGLOC100665',...",RMNP_DP1.00002.001_000.010


### U.S. Census Total County Population Dataset

In [16]:
populationDataFrame = pd.io.json.read_json('../data/county_total_population.Colorado/data.json')
populationDataFrame

,_id,GISJOIN,STATE,STATEFP,STATENH,COUNTY,COUNTYFP,COUNTYNH,1980_total_population,1990_total_population,2000_total_population,2010_total_population
0,{'$oid': '607d4c2708e81eceb6b6e384'},G0800010,Colorado,8,80,Adams County,1,10,245944,265038,363857,441603
1,{'$oid': '607d4c2708e81eceb6b6e385'},G0800030,Colorado,8,80,Alamosa County,3,30,11799,13617,14966,15445
2,{'$oid': '607d4c2708e81eceb6b6e386'},G0800050,Colorado,8,80,Arapahoe County,5,50,293621,391511,487967,572003
3,{'$oid': '607d4c2708e81eceb6b6e387'},G0800070,Colorado,8,80,Archuleta County,7,70,3664,5345,9898,12084
4,{'$oid': '607d4c2708e81eceb6b6e388'},G0800090,Colorado,8,80,Baca County,9,90,5419,4556,4517,3788
...,...,...,...,...,...,...,...,...,...,...,...,...
58,{'$oid': '607d4c2708e81eceb6b6e3bf'},G0801170,Colorado,8,80,Summit County,117,1170,8848,12881,23548,27994
59,{'$oid': '607d4c2708e81eceb6b6e3c4'},G0801190,Colorado,8,80,Teller County,119,1190,8034,12468,20555,23350
60,{'$oid': '607d4c2708e81eceb6b6e3c3'},G0801210,Colorado,8,80,Washington County,121,1210,5304,4812,4926,4814
61,{'$oid': '607d4c2708e81eceb6b6e3c2'},G0801230,Colorado,8,80,Weld County,123,1230,123438,131821,180936,252825


In [17]:
populationLabelsFrame = pd.io.json.read_json('../data/county_total_population.Colorado/fieldLabels.json')
populationLabelsFrame

""


In [18]:
populationGeometryFrame = pd.io.json.read_json('../data/county_total_population.Colorado/linkedGeometry.json')
populationGeometryFrame

,_id,type,geometry,properties,GISJOIN
0,{'$oid': '604964ecfed2e95d6b68b676'},Feature,"{'type': 'Polygon', 'coordinates': [[[-104.961...","{'STATEFP10': '08', 'COUNTYFP10': '001', 'COUN...",G0800010
1,{'$oid': '604964ecfed2e95d6b68b684'},Feature,"{'type': 'Polygon', 'coordinates': [[[-105.457...","{'STATEFP10': '08', 'COUNTYFP10': '003', 'COUN...",G0800030
2,{'$oid': '604964eefed2e95d6b68bb16'},Feature,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...","{'STATEFP10': '08', 'COUNTYFP10': '005', 'COUN...",G0800050
3,{'$oid': '604964eefed2e95d6b68b9c8'},Feature,"{'type': 'Polygon', 'coordinates': [[[-107.128...","{'STATEFP10': '08', 'COUNTYFP10': '007', 'COUN...",G0800070
4,{'$oid': '604964eefed2e95d6b68b83d'},Feature,"{'type': 'Polygon', 'coordinates': [[[-102.747...","{'STATEFP10': '08', 'COUNTYFP10': '009', 'COUN...",G0800090
...,...,...,...,...,...
59,{'$oid': '604964ecfed2e95d6b68b677'},Feature,"{'type': 'Polygon', 'coordinates': [[[-105.924...","{'STATEFP10': '08', 'COUNTYFP10': '117', 'COUN...",G0801170
60,{'$oid': '604964eefed2e95d6b68b840'},Feature,"{'type': 'Polygon', 'coordinates': [[[-105.329...","{'STATEFP10': '08', 'COUNTYFP10': '119', 'COUN...",G0801190
61,{'$oid': '604964eefed2e95d6b68b9c5'},Feature,"{'type': 'Polygon', 'coordinates': [[[-102.779...","{'STATEFP10': '08', 'COUNTYFP10': '121', 'COUN...",G0801210
62,{'$oid': '604964eefed2e95d6b68b9c3'},Feature,"{'type': 'MultiPolygon', 'coordinates': [[[[-1...","{'STATEFP10': '08', 'COUNTYFP10': '123', 'COUN...",G0801230


## Preprocessing
Create Dataframes:

In [ ]:
expected_directories = {
    'covid_county': os.path.join(dataPath, 'covid_county.Colorado'),
    'neon_2d_wind': os.path.join(dataPath, 'neon_2d_wind.Colorado'),
    'neon_single_asp_air_temperature': os.path.join(dataPath, 'neon_single_asp_air_temperature.Colorado'),
    'neon_barometric_pressure': os.path.join(dataPath, 'neon_barometric_pressure.Colorado'),
    'svi_county_GISJOIN': os.path.join(dataPath, 'svi_county_GISJOIN.Colorado')
}
covid_df = data_processing.load_flattened_datasets(
    os.path.join(expected_directories['covid_county'], 'data.json'),
    os.path.join(expected_directories['covid_county'], 'fieldLabels.json'),
    os.path.join(expected_directories['covid_county'], 'linkedGeometry.json'),
    join_on_key='GISJOIN')
wind_df = data_processing.load_flattened_datasets(
    os.path.join(expected_directories['neon_2d_wind'], 'data.json'),
    os.path.join(expected_directories['neon_2d_wind'], 'fieldLabels.json'),
    os.path.join(expected_directories['neon_2d_wind'], 'linkedGeometry.json'),
    join_on_key='site')
air_temp_df = data_processing.load_flattened_datasets(
    os.path.join(expected_directories['neon_single_asp_air_temperature'], 'data.json'),
    os.path.join(expected_directories['neon_single_asp_air_temperature'], 'fieldLabels.json'),
    os.path.join(expected_directories['neon_single_asp_air_temperature'], 'linkedGeometry.json'),
    join_on_key='site')
air_presssure_df = data_processing.load_flattened_datasets(
    os.path.join(expected_directories['neon_barometric_pressure'], 'data.json'),
    os.path.join(expected_directories['neon_barometric_pressure'], 'fieldLabels.json'),
    os.path.join(expected_directories['neon_barometric_pressure'], 'linkedGeometry.json'),
    join_on_key='site')
county_df = data_processing.load_flattened_datasets(
    os.path.join(expected_directories['svi_county_GISJOIN'], 'data.json'),
    os.path.join(expected_directories['svi_county_GISJOIN'], 'fieldLabels.json'),
    os.path.join(expected_directories['svi_county_GISJOIN'], 'linkedGeometry.json'),
    join_on_key='GISJOIN')

print(covid_df.info)
print(wind_df.info)
print(air_temp_df.info)
print(air_presssure_df.info)
print(county_df.info)



## References

## About this Notebook

**Authors:** Kyle Bassignani, Jeff Borgerson, and Christian Westbrook  
**Updated On:** 2021-11-12